<a href="https://colab.research.google.com/github/kakshaycs/RecommendSystem/blob/master/RecomendedSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **A recommender system or a recommendation system (sometimes replacing "system" with a synonym such as platform or engine) is a subclass of information filtering system that seeks to predict the "rating" or "preference" a user would give to an item.They are primarily used in commercial applications bold text** <br>
In this system we have a large no of user vs Song Id data where, users rated more than one song according to their preference and system will find some ralation between more than one user and more than one songs and find a abstract function that can try to predict ration for a song which are not rated by user and according to the rating we will recommend those songs to same user.that may like according to their activity of rating 

In [0]:
# Importing Data

import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from IPython.display import display, clear_output
from random import uniform
import time
from random import randint

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
#loading data
data=pd.read_csv("drive/My Drive/Colab Notebooks/Recommended System/songsDataset.csv")

In [0]:
data.head()

,'userID','songID','rating'
0,0,7171,5
1,0,8637,4
2,0,21966,4
3,0,35821,5
4,0,82446,5


In [0]:
# change column name
data.columns=['userID','songID','rating']

In [0]:
# the no of user and Movie

print("Total no of User:- ",len(set(data['songID'])))
print("Total no of Songs",len(set(data['userID'])))

Total no of User:-  127771
Total no of Songs 200000


In [0]:
# this is the Rating matrix that contain column is user and rows is movies
"""    | movie1 | movie2 | movie3 
 user1  |   5    |    4   |    1
 user2  |   3    |        |    5
 user3  |   5    |    3   |    
  
"""
# the R matrix is boolean matrix that denote user rated that movie or not
print()

There are 127771 users which have rated a least one song with (0-5) rating parameter. And there are 200000 songs that is rated by the users.<br>
This system is using **Matrix factorization** technique to predict songs for particular user which haven't rated those songs.<br>
# Matrix factorization

![alt text](https://cdn-images-1.medium.com/max/1600/1*Zhm1NMlmVywn0G18w3exog.png)

##Matrix factorization is a class of collaborative filtering algorithms used in recommender systems. Matrix factorization algorithms work by decomposing the user-item interaction matrix into the product of two lower dimensionality rectangular matrices.<br>

---

If we use matrix method to store the rating so it take 127771 * 200000 <br>
Total entry is about **25554200000**.which will take about **95 GB** of RAM if we load the whole matrix at a time in RAM. Is very difficult to handle so much large data.
So, there are several aternative to take of these problem<br>

1.   Read in a chunk
2. Process the chunk
3. Save the results of the chunk
4. Repeat steps 1 to 3 until we have all chunk results
5. Combine the chunk results

But as a good knowledge of Alorithm and datastructure we will take care which out using any type of chunk. we store the matrix in Dictionary. Because, given matrix is sparse matrix and we need to handle only 2000000 of user and songs combination so Dictionary will play a important role.

In [0]:
Rating = dict()
for i in range(2000000):
  Rating[(data['userID'][i],data['songID'][i])] = data['rating'][i]
  if i%100000==0:
    clear_output(wait=True)
    print("complete :- ",i/(20000))

complete :-  95.0


We don't know about the features of different songs and what type of songs a user like and dislike. We have only rating by a users <br>
Lets sppose ***user 1*** give 5 start to ***song x*** and <br>
 ***user 2*** also give 5 star to ***song x***<br>,so by only two rating we can conclude that somehow ***user 1*** and ***user 2*** have similar interest.<br>
 And if ***user 1*** rated a ***song y*** to 4 star, So there is large probability that ***user 2*** will also rated ***song y*** to 4 
 

---

So that we assume some features of songs, on the basis of that features a user will predict its rating.<br>
Only asigning features for a song will not help much but we have also some parameters that for a user too.<br>
Both the feature of the songs and parameters of a user can solve our problem.<br>
And one more things both the parameter and the feature are same in numbers but may different in values.<br>
We assigning 5 features for songs and 5 parameters for a user and set a small random value for both features and parameters

example:-           

1.   melody
2.   romance
3. Rock
4. pop
5. classic

***We will assign a random value for each categories in User And Songs Features***


---


Total no of User is 127771 and Songs is 200000 <br> 
So we create two matrix    USER (127771  ,  5) and Song  (200000, 5)

In [0]:
def randomR():
  return round(randint(0,100)/randint(1,100),3)

**Assigning Random variable to the features and parameters**

In [0]:
USERS = np.array([[randomR() for i in range(5)] for i in range(127771)])
SONGS = np.array([[randomR() for i in range(5)] for i in range(200000)])

***OUR GOAL***

We need to find dot product of the features and parameters of a user and a songs to find approximate rating, and accoring to the error we got, we will modify the both features and parameters .<br>
We will repeat this step until the error will minimize<br>
After the minimization of error we got some specific value in features and parameters. And that help to predict the unrated songs.

In [0]:
# define a function that produce dot product of Songs[i],user[j]
def DOT(A,B):
  Temp=0
  for i in range(5):
    Temp+=A[i]*B[i]
  return Temp

## rough work

In [0]:
A=dict()
A['akshay']=1
A['kumar']=2
A['singh']=3

In [0]:
A

{'akshay': 1, 'kumar': 2, 'singh': 3}

In [0]:
for i in A:
  print(i)

akshay
kumar
singh


In [0]:
 127771 * 200000 *4

102216800000

In [0]:
102216800000/(2**30)

95.19681334495544

In [0]:
2**10

1024